### 고령자 보행자 사고는 교차로 또는 횡단보도 인근에서 더 많이 발생한다.

귀무가설(H0): 고령자 보행자 사고량는 교차로 또는 횡단보도 인근 여부와 관련이 없다.
<br>
<br>
대립가설(H1): 고령자 보행자 사고량은 교차로 또는 횡단보도 인근 여부와 관련이 있다.

In [ ]:
# 라이브러리 호출 및 plot 설정
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
from scipy.stats import chi2_contingency
import warnings

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore")